# FACIAL EMOTION CLASSIFICATION

## Step 1: Problem Identification and Description


## Step - 2 Data Collection

## For scraping images to create our dataset we have used 2 sources -
1. Google Image search
2. Instagram

Google image seach was built using chromedriver and follows these steps -
1. Open chrome windows
2. Search google images with the provided query string
3. Click on each image in the results
4. Get the link for the the images
5. After all links are collected for a query, the images are downloaded and saved

Instagram hastag approach followed these steps -
1.
2.
3.

#### Scraping data from google images

In [1]:
import selenium
from selenium import webdriver
import time
import requests
import os
from PIL import Image
import io
import hashlib

In [29]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
             
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                break
        else:
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [30]:
def download_image(folder_path:str,file_name:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        pass

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        folder_path = os.path.join(folder_path,file_name)
        if os.path.exists(folder_path):
            file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        else:
            os.mkdir(folder_path)
            file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
    except Exception as e:
        pass

In [31]:
# Setting the path for chrome driver
CHROME_DRIVER_PATH = os.path.abspath('./') + "\chromedriver.exe"

#### Downloading images for category : "happy"

In [37]:
%%capture
queries = ["happy human face", "happy black man face", "happy white man face", "happy black woman face", "happy white woman face", "happy asian face", "happy indian man face", "happy indian woman face"]
for query in queries:
    wd = webdriver.Chrome(executable_path=CHROME_DRIVER_PATH)
    wd.get('https://google.com')
    search_box = wd.find_element_by_css_selector('input.gLFyf')
    search_box.send_keys(query)
    links = fetch_image_urls(query,100,wd)
    images_path = os.path.abspath('./') + '/images'
    if not os.path.exists(images_path):
        os.mkdir(images_path)
    for i in links:
        persist_image(images_path,query,i)
    wd.quit()

#### Downloading images for category : "sad"

In [35]:
%%capture
queries = ["sad human face", "sad black man face", "sad white man face", "sad black woman face", "sad white woman face", "sad asian face", "sad indian man face", "sad indian woman face"]
for query in queries:
    wd = webdriver.Chrome(executable_path=CHROME_DRIVER_PATH)
    wd.get('https://google.com')
    search_box = wd.find_element_by_css_selector('input.gLFyf')
    search_box.send_keys(query)
    links = fetch_image_urls(query,100,wd)
    images_path = os.path.abspath('./') + '/images'
    if not os.path.exists(images_path):
        os.mkdir(images_path)
    for i in links:
        persist_image(images_path,query,i)
    wd.quit()

#### Downloading images for category : "angry"

In [23]:
%%capture
queries = ["angry human face", "angry black man face", "angry white man face", "angry black woman face", "angry white woman face", "angry asian face", "angry indian man face", "angry indian woman face"]
for query in queries:
    wd = webdriver.Chrome(executable_path=CHROME_DRIVER_PATH)
    wd.get('https://google.com')
    search_box = wd.find_element_by_css_selector('input.gLFyf')
    search_box.send_keys(query)
    links = fetch_image_urls(query,100,wd)
    images_path = os.path.abspath('./') + '/images'
    for i in links:
        persist_image(images_path,query,i)
    wd.quit()

#### Downloading images for category : "disgusted"

In [24]:
%%capture
queries = ["disgusted human face", "disgusted black man face", "disgusted white man face", "disgusted black woman face", "disgusted white woman face", "disgusted asian face", "disgusted indian man face", "disgusted indian woman face"]
for query in queries:
    wd = webdriver.Chrome(executable_path=CHROME_DRIVER_PATH)
    wd.get('https://google.com')
    search_box = wd.find_element_by_css_selector('input.gLFyf')
    search_box.send_keys(query)
    links = fetch_image_urls(query,100,wd)
    images_path = os.path.abspath('./') + '/images'
    if not os.path.exists(images_path):
        os.mkdir(images_path)
    for i in links:
        persist_image(images_path,query,i)
    wd.quit()

#### Downloading images for category : "surprised"

In [25]:
%%capture
queries = ["surprised black man face", "surprised white man face", "surprised black woman face", "surprised white woman face", "surprised indian man face", "surprised indian woman face", "surprised human face","surprised asian face"]
for query in queries:
    wd = webdriver.Chrome(executable_path=CHROME_DRIVER_PATH)
    wd.get('https://google.com')
    search_box = wd.find_element_by_css_selector('input.gLFyf')
    search_box.send_keys(query)
    links = fetch_image_urls(query,100,wd)
    images_path = os.path.abspath('./') + '/images'
    if not os.path.exists(images_path):
        os.mkdir(images_path)
    for i in links:
        persist_image(images_path,query,i)
    wd.quit()

#### Scraping data from instagram

## Step 3: Data exploration

## Step 4: Pre-processing

### Cropping and face centering image data

In [2]:
import cv2
import sys
import os
import matplotlib.pyplot as plt
from mtcnn.mtcnn import MTCNN

In [13]:
class FaceCropper(object):

    def __init__(self):
        self.detector = MTCNN()
    
    def generate(self, image_path,prefix,output_path):
        img = cv2.imread(image_path)
        if (img is None):
            return 0
        faces = self.detector.detect_faces(img)
        for i, face in enumerate(faces): # iterate through all the faces found
            box=face['box']
            if box !=[]:
                confidence = face['confidence']
                if(confidence < 0.9):
                    continue
                # return all faces found in the image
                box[0]= 0 if box[0]<0 else box[0]
                box[1]= 0 if box[1]<0 else box[1]
                
                cropped_img=img[box[1]: box[1]+box[3],box[0]: box[0]+ box[2]]
                try:
                    cv2.imwrite(output_data_path + prefix, cropped_img)
                except Exception as e:
                    pass

In [5]:
%%capture
detecter = FaceCropper()
categories = ["happy", "sad", "angry", "disgusted","surprised"]
data_folder = "/data/"
for category in categories:
    input_data_path = os.path.abspath('./') + data_folder + category
    output_data_path = os.path.abspath('./') + "/filtered_data"

    if not os.path.exists(output_data_path):
        os.mkdir(output_data_path)
    output_data_path = output_data_path + "/" + category + "/"
    if not os.path.exists(output_data_path):
        os.mkdir(output_data_path)
    for filename in os.listdir(input_data_path):
        image_path = input_data_path + "/" + filename
        detecter.generate(image_path,filename,output_data_path)
        break

## Step 5: Data analysis with deep learning

## Step 6: Numerical results

## References

1. https://medium.com/@wwwanandsuresh/web-scraping-images-from-google-9084545808a2
2. https://gist.github.com/tilfin/98bbba47fdc4ac10c4069cce5fabd834
3. https://github.com/stanford-traffic-sign/stanford-cs230-traffic-sign